In [1]:
import arcpy

In [2]:
#Define the SDE connection file path
sde_con_path = 'C:/Projects'

In [8]:
#Define the SDE connection file name
sde_con_file = 'your_sde_connection_name_here.sde'

In [9]:
sde_con = sde_con_path + '/' + sde_con_file

In [5]:
database = 'your_database_here'

In [6]:
schema = 'dbo'

In [28]:
#If there is no existing SDE connection file, then make one
arcpy.CreateDatabaseConnection_management(out_folder_path = sde_con_path,
                                          out_name = sde_con_file,
                                          account_authentication = 'DATABASE_AUTH',
                                          username = 'your_username_here',
                                          password = 'your_password_here',
                                          instance = 'your_server_name_here', 
                                          database = database,
                                          database_platform = 'SQL_SERVER')

In [10]:
#Set the workspace to the appropriate SDE connection
arcpy.env.workspace = sde_con

In [11]:
#Define the database and schema
db_schema = database + '.' + schema + '.'

In [13]:
#Retrieve the list of feature classes that are in the current workspace, convert the list elements to a 
#string and make the names lower case
gdb_fcs = [str(a).lower() for a in arcpy.ListFeatureClasses()]

In [14]:
#Define all the feature classes that may be edited
edit_fcs = ['TBL_DPR_DoITT_GREENROOF', 'TBL_NonActive_DoITT_Greenroof', 'TBL_CapitalProperty',
            'TBL_DailyTasks_Sites', 'TBL_CNA_STRUCTURES', 'TBL_PARKSGIS_ALLSITES', 'TBL_PARKSGIS_GREENROOF',
            'TBL_PARKSGIS_STRUCTURE', 'TBL_AMPS_BUILDINGS', 'TBL_DM_BUILDINGS', 'TBL_DM_GISALLSITES',
            'TBL_OPS_COMPLETEDACTIVATIONWOS', 'TBL_OPS_DFACTIVATION', 'TBL_LaborRel_LactationSites',
            'TBL_PPS_ALLSITES', 'TBL_PPS_FACILITYPOINTS']

In [15]:
#Append the database name and schema and make all the feature class names lower case
edit_fcs = [(db_schema + f).lower() for f in edit_fcs]

In [16]:
#Define the fields that are being deleted
delete_fields = ['TypeCategory', 'ACRES', 'AMPSSync', 'AMPSSyncDate', 'BATHROOM',
                 'COMFORTSTATION', 'ComfortStation_1', 'CommunityBoard', 'CouncilDistrict',
                 'CSCount', 'District', 'EstSqFeet', 'FEMARisk', 'GSGroupProject', 
                 'Map____Hagstrom_', 'Map_Grid__Hagstrom_', 'NAME', 'NYS_ASSEMBLY', 'NYS_SENATE',
                 'PAD_addresses', 'PERMITDISTRICT', 'Phone', 'PIPCreated', 'PIPZipCode', 'PRECINCT',
                 'RETIRED', 'RETIREDDATE', 'Safety_Index', 'sBATHROOM', 'SIGNNAME', 'Sort', 'sRETIRED',
                 'STRUCTURETYPE', 'Sub_Properties_Rated', 'Sub_Property', 'SubPropertyName', 'US_CONGRESS',
                 'ZIPCODE']

In [17]:
#Define the fields that should be moved to null values instead of being deleted.
#Just here for tracking purposes.
null_fields = ['Address', 'Historic']

In [18]:
#Make all the field names lower case
delete_fields = [d.lower() for d in delete_fields]

In [25]:
#Open/create a log file in append mode in the same directory as the python script
field_audit = open(r'C:\Projects\field_audit.txt', 'a')

#Iterate through the feature classes in the geodatabase
for g in gdb_fcs:
    #If the feature class from the geodatabase is in the list of feature classes being edited then 
    if g in edit_fcs:
        field_audit.write('\n Feature_Class: {} \n'.format(g))
        #Get all the field names in the matching feature class and convert the names to lower case strings
        gdb_fields = [str(f.name).lower() for f in arcpy.ListFields(dataset = g)]
        
        #Iterate through the fields that have been flagged for deletion
        for n in null_fields:
            #If the fields to be nulled are in the fields of the feature class
            if n in gdb_fields:
                #Print a message indicating that they will be nulled
                field_audit.write('Not deleting column {} from {}, but the data source is changing. \n'.format(d, g))
        
        for d in delete_fields:
            #If the fields to be deleted are in the fields of the feature class
            if d in gdb_fields:            
                #Delete the fields from feature class, in place. If we wanted to, we could copy the 
                #feature class to a new feature class or in memory and then make the edits, but this 
                #doesn't seem particularly likely to encounter any exceptions.
                try:
                    arcpy.DeleteField_management(in_table = g, drop_field = d)
                    #Print a message indicating the deletion
                    field_audit.write('Deleted column {} from {}. \n'.format(d, g))
                
                except Exception as e:
                    field_audit.write('Could not delete column {} from {}, exception below. \n'.format(d, g))
                    field_audit.write('Exception: {} \n'.format(e))
field_audit.close()